In [6]:
import numpy as np
import networkx as nx
import bct as bct
from pathlib import Path
import re
import pandas as pd
from tqdm import tqdm
from pandas import read_csv, read_excel
from skimage import io

In [7]:
graphs_path = Path('matt_preds_graphs')
graphs_paths = list(graphs_path.glob('*warped_radii.pickle'))
graphs_paths = sorted([x.as_posix() for x in graphs_paths])
graphs_paths = [x for x in graphs_paths if '108' not in x]
graphs_paths = [x for x in graphs_paths if '-' in x]
len(graphs_paths)

260

In [11]:
#df = read_excel(re.sub('matt_preds_graphs','matt_ev_dst',re.sub('.pickle','.xlsx',graphs_paths[1])), header=None)
#print(df)

In [ ]:
#df.keys()

In [ ]:
#print(np.mean(df[0]))
#print(np.std(df[0]))

reference for bct functions:
<br>
https://htmlpreview.github.io/?https://github.com/aestrivex/bctpy/blob/master/function_reference.html

In [38]:
df = pd.DataFrame(columns=["transitivity",
                           "assortativity",
                           "modularity",
                           "density",
                           "transitivity_0001",
                           "assortativity_0001",
                           "modularity_0001",
                           "effeciency",
                           "assortativity_delta",
                           "subject",
                           "treatment",
                           "wavelength",
                           "power_per",
                           "run"])

xls = pd.ExcelFile('TBI_STIM_metalog.xlsx')
df2 = {}
for sheet_name in xls.sheet_names:
    df2[sheet_name] = xls.parse(sheet_name)



for i in tqdm(range(len(graphs_paths))[::-1]):
    res = {}
    graph = nx.read_gpickle(graphs_paths[i])
    #img = io.imread(re.sub('matt_preds_graphs','matt_ev_dst',re.sub('_warped_radii.pickle','_ev_dst.tif',graphs_paths[i])))
    #neurons = np.load(re.sub('matt_preds_graphs','matt_preds',re.sub('_warped_radii.pickle','_mean.npy',graphs_paths[i])))
    #df3 = img[np.argmax(neurons,axis=0)==2]                   
    for sheet_name in xls.sheet_names:
            if re.sub('matt_preds_graphs/','',re.sub('_warped_radii.pickle','',graphs_paths[i])).split('-')[1] in df2[sheet_name].values and re.sub('matt_preds_graphs/','',re.sub('_warped_radii.pickle','',graphs_paths[i])).split('-')[0].split('_')[1]in sheet_name:
                subj = sheet_name
                if subj in ["TBI07_3D","TBI11_3D","TBI22_3D","TBI31_3D","TBI38_3D","SHAM09_3D","SHAM12_3D","SHAM23_3D","SHAM32_3D",'TBI38_3D','TBI6_3D','SHAM7_3D','TBI43_3D','TBI45_3D','SHAM53_3D','SHAM56_3D','SHAM55_3D']:
                    gender = 'male'
                else:
                    gender = 'female'
                treatment = re.sub('SHA','SHAM',subj[0:3])
                _tmp = df2[subj].loc[df2[subj]['CHECK WATER'] == re.sub('matt_preds_graphs/','',re.sub('_warped_radii.pickle','',graphs_paths[i])).split('-')[1]]
                if _tmp['Unnamed: 12'].iloc[0] == 'raster':
                    wavelength = _tmp['Unnamed: 11'].iloc[0]
                    power_per = _tmp['Unnamed: 10'].iloc[0]
                    Gcc = sorted(nx.connected_components(graph), key=len, reverse=True)
                    G0 = graph.subgraph(Gcc[0])
                    graph_adjacency_matrix = nx.adjacency_matrix(G0,weight = 'radii').toarray()
                    graph_adjacency_matrix_0001 = nx.adjacency_matrix(G0,weight = 'radii_0001').toarray()
                    res["subject"] = subj
                    res["gender"] = gender
                    res["treatment"] = treatment
                    res["wavelength"] = wavelength
                    res["power_per"] = power_per
                    #res["EV_dst"] = np.mean(df3)
                    #res["EV_dst_std"] = np.std(df3)
                    res["transitivity"] = bct.transitivity_wu(graph_adjacency_matrix)
                    res["assortativity"] = bct.assortativity_wei(graph_adjacency_matrix)
                    res["modularity"] = bct.modularity_und(graph_adjacency_matrix)[1]
                    res["density"] = bct.density_und(graph_adjacency_matrix)[0]
                    res["transitivity_0001"] = bct.transitivity_wu(graph_adjacency_matrix_0001)
                    res["assortativity_0001"] = bct.assortativity_wei(graph_adjacency_matrix_0001)
                    res["modularity_0001"] = bct.modularity_und(graph_adjacency_matrix_0001)[1]
                    res["run"] = re.sub('matt_preds_graphs/','',re.sub('_warped_radii.pickle','',graphs_paths[i])).split('-')[1]
                    
                    graph_adjacency_matrix = nx.adjacency_matrix(graph,weight = 'weight').toarray()
                    res["effeciency"] = bct.efficiency_wei(graph_adjacency_matrix)
                    graph_adjacency_matrix = nx.adjacency_matrix(graph,weight = 'delta').toarray()
                    res["assortativity_delta"] = bct.assortativity_wei(graph_adjacency_matrix)
                    df.loc[re.sub('_warped_single_radii_matched.pickle','',re.sub('matt_preds_graphs/','',graphs_paths[i]))] = res

 35%|███▍      | 90/260 [00:12<00:02, 67.79it/s]/lustre06/project/6023374/rozakmat/monai3.8/lib/python3.8/site-packages/bct/algorithms/clustering.py:721: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(cyc3, axis=0) / np.sum(K * (K - 1), axis=0)
/lustre06/project/6023374/rozakmat/monai3.8/lib/python3.8/site-packages/bct/algorithms/core.py:139: RuntimeWarning: invalid value encountered in double_scalars
  r = (term1 - term2) / (term3 - term2)
/lustre06/project/6023374/rozakmat/monai3.8/lib/python3.8/site-packages/bct/algorithms/core.py:136: RuntimeWarning: invalid value encountered in double_scalars
  term1 = np.sum(stri * strj) / K
/lustre06/project/6023374/rozakmat/monai3.8/lib/python3.8/site-packages/bct/algorithms/core.py:137: RuntimeWarning: invalid value encountered in double_scalars
  term2 = np.square(np.sum(.5 * (stri + strj)) / K)
/lustre06/project/6023374/rozakmat/monai3.8/lib/python3.8/site-packages/bct/algorithms/core.py:138: RuntimeWarning: inva

In [37]:
re.sub('matt_preds_graphs/','',re.sub('_warped_radii.pickle','',graphs_paths[i])).split('-')[1]

'XYZres298'

In [39]:
df

,transitivity,assortativity,modularity,density,transitivity_0001,assortativity_0001,modularity_0001,effeciency,assortativity_delta,subject,treatment,wavelength,power_per,run
Feb52021_6-XYZres282_warped_radii.pickle,1.493700,0.550412,0.738058,0.020868,1.515966,0.557259,0.740272,0.910805,0.492196,TBI6_3D,TBI,552,20,XYZres282
Feb52021_6-XYZres280_warped_radii.pickle,2.329075,-0.083276,0.497460,0.213235,0.620794,-0.002085,0.498837,0.755165,0.037854,TBI6_3D,TBI,458,20,XYZres280
Feb52021_6-XYZres279_warped_radii.pickle,1.382401,0.596700,0.726839,0.023520,1.386138,0.608117,0.724424,1.099433,0.293788,TBI6_3D,TBI,458,20,XYZres279
Feb52021_6-XYZres278_warped_radii.pickle,1.286125,0.660644,0.783081,0.011233,1.297491,0.661240,0.796064,1.344236,0.438754,TBI6_3D,TBI,458,20,XYZres278
Feb52021_6-XYZres277_warped_radii.pickle,0.614458,0.173062,0.521510,0.161765,0.317124,0.173884,0.475747,0.678563,0.361681,TBI6_3D,TBI,458,20,XYZres277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
06162021_45-XYZres297_warped_radii.pickle,1.857678,0.511852,0.749523,0.014778,1.834463,0.515181,0.782266,1.202433,0.145109,TBI45_3D,TBI,458,20,XYZres297
06162021_45-XYZres295_warped_radii.pickle,1.451339,0.261917,0.760865,0.017283,1.424531,0.269589,0.767557,1.035675,0.209181,TBI45_3D,TBI,458,5,XYZres295
06162021_45-XYZres294_warped_radii.pickle,2.011479,0.590951,0.773804,0.020645,1.801404,0.593086,0.762340,0.896727,0.303369,TBI45_3D,TBI,458,5,XYZres294
06162021_45-XYZres290_warped_radii.pickle,2.630400,-0.743370,0.282663,0.833333,1.984263,-0.451138,0.209830,1.785732,-1.000000,TBI45_3D,TBI,458,20,XYZres290


In [ ]:
neurons.shape

In [ ]:
graphs_paths[i]

In [ ]:
re.sub('matt_preds_graphs','matt_preds',re.sub('_warped_radii.pickle','_mean.npy',graphs_paths[i]))

In [ ]:
re.sub('matt_preds_graphs','matt_ev_dst',re.sub('_warped_radii.pickle','.csv',graphs_paths[i]))

In [40]:
df.to_excel('bct_matt_brain.xlsx')

In [ ]:
df

In [ ]:
re.sub('matt_preds_graphs/','',re.sub('_warped_radii.pickle','',graphs_paths[i]))

In [ ]:
graph_adjacency_matrix = nx.adjacency_matrix(graph,weight = 'weight').toarray()
graph_adjacency_matrix_0001 = nx.adjacency_matrix(graph,weight = 'weight_0001').toarray()

In [ ]:
bct.efficiency_wei(graph_adjacency_matrix)